In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

**Prepare**

In [ ]:
import numpy as np
from numpy import genfromtxt
from numpy import asarray
import math
import copy
import os
from PIL import Image 
import cv2

patch_size = 64 #input = 64x64
label_size = 128 #output = 128x128

#get RGGB bayer image
def bayer_reverse(img):
    height,width,c = img.shape;
    tmp = np.zeros([height,width]);
    for i in range( height ):
        for j in range( width ):
            if i % 2 == 0 :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][0];#R
                else:
                    tmp[i][j] = img[i][j][1];#G
            else :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][1];#G
                else:
                    tmp[i][j] = img[i][j][2];#B

    return tmp;

#split image to prepare the train set
def split(img,name,dir_path):
    height,width,c = img.shape;
    # print(img.shape)
    count = 0;
    for i in range(0,height,30):
        for j in range(0,width,30):
            if( i + label_size < height and j + label_size < width ):
                tmp = np.zeros([label_size,label_size,3]);
                tmp2 = np.zeros([label_size,label_size,3]);
                
                tmp = img[ i : i + label_size, j : j + label_size,:];
                                
                path = os.path.join(dir_path,'label/'+name.split('.')[0] +'_'+str(count)+'.png')                
                im = Image.fromarray(tmp)               
                tmp2[:,:,0] = tmp[:,:,2]
                tmp2[:,:,1] = tmp[:,:,1]                
                tmp2[:,:,2] = tmp[:,:,0]                
                cv2.imwrite(path,tmp2)

                zoom = im.resize((patch_size,patch_size)) 
                zoom2 = np.zeros([patch_size,patch_size,3]);
                # gray =  np.zeros([patch_size,patch_size]);
                
                zoom = np.array(zoom)
                zoom2[:,:,0] = zoom[:,:,2]
                zoom2[:,:,1] = zoom[:,:,1]                
                zoom2[:,:,2] = zoom[:,:,0]
                
                # zoom2 = bayer_reverse(zoom2)
                #gray = gray/255                
                path = os.path.join(dir_path,'patch/'+name.split('.')[0] +'_'+str(count)+'.png')
                # im = Image.fromarray(zoom2)           
                cv2.imwrite(path, zoom2)

                count = count + 1

def main():
    path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
    if not os.path.exists(os.path.join(path,'patch')):
        os.makedirs(os.path.join(path,'patch'))
    
    if not os.path.exists(os.path.join(path,'label')):
        os.makedirs(os.path.join(path,'label'))
    
    dataset_path = os.path.join(path,'origin')
    entries = os.listdir(dataset_path)
    for entry in entries:
        print(entry)
        img_path = dataset_path + '/' + entry
        img = Image.open(img_path)
        img = np.array(img)   
        split(img,entry,path)



if __name__ == '__main__':
    main()


**Training**

In [ ]:
import numpy as np
from tensorflow.keras import layers
from keras.preprocessing import image
import tensorflow as tf 
from keras.models import Model,load_model
from keras.utils import to_categorical
import os
import keras
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Lambda
from keras.optimizers import Adam
from keras import backend as K
import random
from PIL import Image 
from random import shuffle

batch_sz = 16
oti = 'adam'
lr = 0.001
e_num = 20


# def main():

train_image = []
train_label = []

dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
patch_path = os.path.join(dir_path,'patch')
entries = os.listdir(patch_path)
for entry in entries:
  im = image.load_img(patch_path+'/'+entry, target_size = (64, 64))
  img = image.img_to_array(im)    
  # img = img[:,:,0]    
  # img = img[:,:,np.newaxis]           # Modify here!!!
  train_image.append(img)
train_image= np.stack(train_image)

print(train_image.shape)# (x,128,128,1)
  

label_path = os.path.join(dir_path,'label')
entries = os.listdir(label_path)
for entry in entries:
  im = image.load_img(label_path+'/'+entry, target_size = (128, 128))
  img = image.img_to_array(im)
  train_label.append(img)
train_label = np.stack(train_label)

print(train_label.shape)# (x,256,256,3)



index = [i for i in range(train_image.shape[0])]
shuffle(index)
train_image = train_image[index,:,:,:];
train_label = train_label[index,:,:,:];

# model.save(os.path.join(dir_path,'model.h5'))

# if __name__ == '__main__':
#   main()


In [ ]:
############################# Model Structure ################################################
def create_model():
  inputs = keras.Input(shape=(None,None,3))

  ##Subpixel Construction
  sub_layer_2 = Lambda(lambda x:tf.nn.space_to_depth(x,2)) 
  init = sub_layer_2(inputs=inputs)


  ##Learning Residual (DCNN)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters=64,
                      kernel_size = 3, 
                      strides = 1,  # 2
                      padding = 'same', 
                      input_shape = (None,None,3))(init)

  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Residual Block
  for i in range(6):
    Conv1 = keras.layers.SeparableConv2D(filters=64,
                        kernel_size = 3, 
                        strides = 1,  # 2
                        padding = 'same',
                        input_shape = (None,None,64))(x)
    
    PReLu = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(Conv1)
    Conv2 = keras.layers.SeparableConv2D(filters=64,
                        kernel_size = 3, 
                        strides = 1,  # 2
                        padding = 'same',
                        input_shape = (None,None,64))(PReLu)
    
    
    x = keras.layers.Add()([Conv2,x])
    x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters=64,
                      kernel_size = 3, 
                      strides = 1,  # 2
                      padding = 'same', 
                      input_shape = (None,None,1))(x)

  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Conv 3x3x64x48
  x = keras.layers.SeparableConv2D(filters=48,
                      kernel_size = 3, 
                      strides = 1,  
                      padding = 'same',                      
                      input_shape = (None,None,64))(x)

  ###########Learning Residual (DCNN)############

  ##Recovery From Subpixel
  sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,4)) 
  Residual_Output = sub_layer(inputs=x)


  ##Initial Prediction
  R = Lambda(lambda x: x[:,:,:,0])(init)
  G = Lambda(lambda x: x[:,:,:,1:3])(init)
  G = Lambda(lambda x: K.mean(x, axis=3))(G)
  B = Lambda(lambda x: x[:,:,:,3])(init)
  # print(init.shape)
  # print(R.shape)
  # print(G.shape)
  # print(B.shape)
  R = Lambda(lambda x: tf.expand_dims(x, -1))(R)
  G = Lambda(lambda x: tf.expand_dims(x, -1))(G)
  B = Lambda(lambda x: tf.expand_dims(x, -1))(B)

  #rgb = tf.keras.backend.stack((R, G,B),axis =  3)
  # print(R.shape)
  rg = keras.layers.Concatenate(axis = 3)([R , G])
  rgb = keras.layers.Concatenate(axis = 3)([rg,B])
  # print(rgb.shape)
  Coarse_Output = keras.layers.UpSampling2D(size=(4, 4))(rgb)

  ## + 
  outputs = keras.layers.Add()([Residual_Output,Coarse_Output])

  model = keras.Model(inputs=inputs, outputs=outputs, name="JDMSR_model")  
  return model

In [ ]:
model = create_model()
model.compile(optimizer=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss = 'mean_squared_error', metrics = ['accuracy'])

#histories = Histories()
checkpoint = ModelCheckpoint(os.path.join(dir_path,'model.hdf5'),verbose=1, monitor='val_loss', 
                              save_best_only=True,save_weights_only=True)
rrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='min', min_lr=0.0000002)

history = model.fit(train_image, train_label, epochs=e_num, batch_size=batch_sz,verbose=1,
            validation_split = 0.1,callbacks=[checkpoint,rrp],shuffle = True)

In [ ]:
# Plotting
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy - 2')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('model_accuracy2.png')
plt.show()

plt.plot()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss - 2')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig('model_loss2.png')
plt.show()

**Validation**

In [ ]:
from keras.models import load_model
from keras.layers import Lambda
from keras.preprocessing import image
import keras
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
from keras import backend as K
import os
import math
import shutil
oti = 'adam'


dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
model = create_model()
model.load_weights(os.path.join(dir_path,'model.hdf5'))


output_path = os.path.join(dir_path, 'output')
output_path2 = os.path.join(dir_path, 'output2')

if os.path.exists(output_path):
    shutil.rmtree(output_path)
os.makedirs(output_path)

if os.path.exists(output_path2):
    shutil.rmtree(output_path2)
os.makedirs(output_path2)

input_path = os.path.join(dir_path, 'Set14')
entries = os.listdir(input_path)

print("--------  Start Validation --------")
for entry in entries:
    # Test Image
    path = input_path+'/'+entry
    test_image = Image.open(path)
    
    if not test_image.size[0]%2 == 0:
      test_image = test_image.resize((test_image.size[0]-1, test_image.size[1]), Image.BILINEAR)
    if not test_image.size[1]%2 == 0:
      test_image = test_image.resize((test_image.size[0], test_image.size[1]-1), Image.BILINEAR)

    print(test_image.size)
    original_image = test_image.resize((test_image.size[0]*2, test_image.size[1]*2), Image.BILINEAR)
    path = output_path2+'/'+entry
    original_image.save(path)

    print(test_image.size)
    test_image = np.array(test_image)
    print(test_image.shape)
    test_image = test_image[np.newaxis,:,:]
    print(test_image.shape)

    out = model.predict(test_image)
    out = out[0]   
    out = image.array_to_img(out)
    path = output_path+'/'+entry
    out.save(path)
print("-------- Finish Validation  --------")

--------  Start Validation --------
(276, 276)
(276, 276)
(276, 276, 3)
(1, 276, 276, 3)
(500, 362)
(500, 362)
(362, 500, 3)
(1, 362, 500, 3)
(250, 360)
(250, 360)
(360, 250, 3)
(1, 360, 250, 3)
(512, 512)
(512, 512)
(512, 512, 3)
(1, 512, 512, 3)
(528, 656)
(528, 656)
(656, 528, 3)
(1, 656, 528, 3)
(500, 480)
(500, 480)
(480, 500, 3)
(1, 480, 500, 3)
(512, 512)
(512, 512)
(512, 512, 3)
(1, 512, 512, 3)
(720, 576)
(720, 576)
(576, 720, 3)
(1, 576, 720, 3)
(352, 288)
(352, 288)
(288, 352, 3)
(1, 288, 352, 3)
(512, 512)
(512, 512)
(512, 512, 3)
(1, 512, 512, 3)
(352, 288)
(352, 288)
(288, 352, 3)
(1, 288, 352, 3)
(768, 512)
(768, 512)
(512, 768, 3)
(1, 512, 768, 3)
(512, 512)
(512, 512)
(512, 512, 3)
(1, 512, 512, 3)
(586, 390)
(586, 390)
(390, 586, 3)
(1, 390, 586, 3)
-------- Finish Validation  --------


**Performance**

In [ ]:
import cv2
import numpy as np
import math
import os
from google.colab.patches import cv2_imshow
from skimage.measure import compare_ssim

def calculate_psnr(original, contrast):
    mse = np.mean((original - contrast) ** 2)
    if mse == 0:
        return 100
    max_value = 255.0
    return 20 * math.log10(max_value / math.sqrt(mse))
    

path = 'drive/My Drive/Colab Notebooks/undergraduate_project' 
input_path = os.path.join(path,'output')
output_path = os.path.join(path,'output2')
entries = os.listdir(input_path)
count = 0
total_psnr = 0.
total_ssim = 0.

for entry in entries:
  img1 = cv2.imread(os.path.join(input_path,entry))
  img2 = cv2.imread(os.path.join(output_path,entry))
  # img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]), interpolation=cv2.INTER_CUBIC)
  psnr = calculate_psnr(img1,img2)
  print("PSNR-{0}: {1:.10f}dB".format(entry,psnr))
  ssim = compare_ssim(img1, img2, multichannel=True)
  print("SSIM-{0}: {1:.10f}".format(entry,ssim))
  total_psnr += psnr
  total_ssim += ssim
  count += 1
print(count)
# cv2_imshow(img1)
# cv2_imshow(img2)

total_psnr = total_psnr / count
total_ssim = total_ssim / count
print("\n=====================================")
print("Average PSNR:{:.10f}".format(total_psnr))
print("Average SSIM:{:.10f}".format(total_ssim))




PSNR-face.png: 28.3243303750dB
SSIM-face.png: 0.7639348999
PSNR-flowers.png: 28.0414611060dB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


SSIM-flowers.png: 0.8585437544
PSNR-comic.png: 28.8671512143dB
SSIM-comic.png: 0.9386269585
PSNR-bridge.png: 28.8560914818dB
SSIM-bridge.png: 0.9170089340
PSNR-ppt3.png: 27.4606706123dB
SSIM-ppt3.png: 0.8509819235
PSNR-baboon.png: 27.9018888841dB
SSIM-baboon.png: 0.7656595234
PSNR-lenna.png: 30.0393172850dB
SSIM-lenna.png: 0.8992952384
PSNR-barbara.png: 28.3207661045dB
SSIM-barbara.png: 0.8946657592
PSNR-coastguard.png: 28.1006749768dB
SSIM-coastguard.png: 0.9458464889
PSNR-man.png: 28.4855418540dB
SSIM-man.png: 0.7856137805
PSNR-foreman.png: 27.9806757549dB
SSIM-foreman.png: 0.9367705249
PSNR-monarch.png: 29.9128844973dB
SSIM-monarch.png: 0.9629333332
PSNR-pepper.png: 28.0542032723dB
SSIM-pepper.png: 0.8059124542
PSNR-zebra.png: 29.2695414434dB
SSIM-zebra.png: 0.9286183603
14

Average PSNR:28.5439427758
Average SSIM:0.8753151381
